# Parking-Lot-Design
## Features
- Designed a Parking Lot Management System from scratch applying OOPs concepts in Python.
- Simulated a real-life scenario of the randomized system to test the functionality of the system.
- Comprehensive Log Generation for monitoring the scenario.

## Languages Used
- Python

## Assumptions and Constraints
- The parking layout envisioned was a multilevel Parking Area. A parking Area having multiple floors (levels) and each floor having slots like grid divided into rows and columns.
- We assumed that different vehicles would take different area to park depending on their dimensions. 
  - Motorcycle would take 1 Grid Cell
  - Car would take 2 Grid Cells
  - Bus would take 3 Grid Cells
- Vehicles can **only** be parked in one single row of parking space. As in horizontally only. Since between different rows would be a space, so that vehicles could move in and out of parking area.


In [17]:
# Importing relevant libraries
import enum
from abc import ABC, abstractmethod
import random
import time
import sys

In [20]:
# Enumerator Class to Define Vehicle Sizes
class VehicleSize(enum.Enum):
    MOTORCYCLE = 0
    CAR = 1
    BUS = 2

# Enumerator Class to Define Vehicle Prices
class ParkingPricingPerDay(enum.Enum):
    MOTORCYCLE = 100
    CAR = 150
    LARGE = 200


# Vehicle Abstract Class
class Vehicle(ABC):
    def __init__(self, vehicle_type, license_plate):
        self.vehicle_type = vehicle_type
        self.license_plate = license_plate
        self.slot_location = None
        self.total_payable_cost = 0

    def clear_slot(self):
        self.slot_location = None

    def assign_slot(self, row, col, level):
        self.slot_location = (row, col, level)

    @abstractmethod
    def compute_cost(self, days):
        pass

# Motorcycle Class which inherits from Vehicle Abstract Class
class Motorcycle(Vehicle):
    def __init__(self, license_plate):
        super(Motorcycle, self).__init__(VehicleSize.MOTORCYCLE, license_plate)

    def compute_cost(self, days):
        return days * ParkingPricingPerDay.MOTORCYCLE

# Car Class which inherits from Vehicle Abstract Class
class Car(Vehicle):
    def __init__(self, license_plate):
        super(Car, self).__init__(VehicleSize.CAR, license_plate)

    def compute_cost(self, days):
        return days * ParkingPricingPerDay.CAR

# Bus Class which inherits from Vehicle Abstract Class
class Bus(Vehicle):
    def __init__(self, license_plate):
        super(Bus, self).__init__(VehicleSize.BUS, license_plate)

    def compute_cost(self, days):
        return days * ParkingPricingPerDay.LARGE

"""
class ParkingLot  
Description: Made for parking management of vehicles in the parking lot.
Has properties/ attributes:
- rows => Defines the number of rows of car parking on a level
- columns => Defines the number of columns of car parking on a particular row
- level => Number of floors the parking lot spans
Has the following methods:
- check_availability(self, parking_row, vehicle_type)
Checks availability of slots for a particular vehicle type in a parking row.
- park_vehicle(self, vehicle)
Parks a vehicle in the available slot in the parking lot.
- remove_vehicle(self, vehicle)
Removes a vehicle from a parking slot, if parked in the lot.
- show_current_status(self)
Gives a detailed overview of the entire parking lot, on which slots are available and which are empty across various levels and rows.
"""
class ParkingLot:
    # Default Constructor
    def __init__(self, rows, columns, levels) -> None:
        self.rows = rows
        self.columns = columns
        self.levels = levels
        self.parking_lot = {
            level: {row: [
                {
                    "available": True,
                    "vehicle": None
                }
                for column in range(columns)]
                for row in range(rows)}
            for level in range(levels)
        }

    # Checks availability of slots for a particular vehicle type in a parking row.
    def check_availability(self, parking_row, vehicle_type):
        for column in range(self.columns - int(vehicle_type)):
            temp = parking_row[column]
            if not temp["available"]:
                # column += temp["vehicle"].vehicle_type.value
                continue
            else:
                iterator = column + 1
                flag = True
                while iterator <= (column + vehicle_type):
                    flag = flag and parking_row[iterator]["available"]
                    if not flag:
                        break
                    iterator += 1
                if flag:
                    return column
        return -1

    def park_vehicle(self, vehicle):
        flag = False
        vehicle_type = vehicle.vehicle_type.value
        levels = [i for i in range(self.levels)]
        random.shuffle(levels)
        rows = [i for i in range(self.rows)]
        random.shuffle(rows)
        for level in levels:
            for row in rows:
                parking_row = self.parking_lot[level][row]
                allotted_slot = self.check_availability(parking_row, vehicle_type)
                if allotted_slot != -1:
                    vehicle.slot_location = (level, row, allotted_slot)
                    flag = True
                    self.parking_lot[level][row][allotted_slot]["vehicle"] = vehicle
                    max_occupied_slot = allotted_slot + vehicle_type
                    while allotted_slot <= max_occupied_slot:
                        parking_row[allotted_slot]["available"] = False
                        allotted_slot += 1
                    print("." * 30)
                    print(
                        "Vehicle " + vehicle.vehicle_type.name + " with " + vehicle.license_plate +
                        " is now occupying the slot "
                        + str(vehicle.slot_location) +
                        " to " + str((level, row, max_occupied_slot)))
                    print("Parked Successfully.")

                    print("." * 30 + "\n")
                    return flag

        print("." * 30)
        print("Vehicle " + vehicle.vehicle_type.name + " with " + vehicle.license_plate +
              " could not be parked.")
        print("." * 30 + "\n")
        return flag

    def remove_vehicle(self, vehicle):
        slot_location = vehicle.slot_location
        vehicle_type = vehicle.vehicle_type
        for i in range(slot_location[2], slot_location[2] + vehicle_type.value + 1):
            obj = self.parking_lot[slot_location[0]][slot_location[1]][i]
            obj["available"] = True
            obj["vehicle"] = None
        print("*" * 30)
        print("Vehicle " + vehicle.vehicle_type.name + " with " + vehicle.license_plate + " occupying the slot "
              + str(slot_location) +
              " to " + str((slot_location[0], slot_location[1], slot_location[2] + vehicle_type.value))
              + " is removed from parking")
        print("*" * 30 + "\n")

    def show_current_status(self):
        print("###" * 20)
        print("Current Status of Parking Lot"
              "")
        print("###" * 20)
        for level in range(self.levels):
            print("-" * 30 + "At level " + str(level) + ":" + "-" * 30)
            for row in range(self.rows):
                print("*" * 80)
                print("At row " + str(row) + ":")
                obj = self.parking_lot[level][row]
                print(obj)
                print("*" * 80)
            print()
        print("###" * 20)


"""
class TestParkingLot
Description: Made to test the functionaity of the parking lot class and simulating a real life parking lot situation.
Properties/ Attributes:
- vehicles_count: Keeps track of total number of vehicles .
- distribution_of_vehicles: Dictionary of the input vehicle types and count.
- rows: rows of parking lot
- columns: columns of parking lot
- levels: levels of parking lot
- vehicle_list: stores a list of vehicles in the simulation
- parked: boolean array to keep track of if the vehicles are parked or not.
- parking_lot: An object of "ParkingLot" class of dimension (rows, columns, height).
Methods:
- generate_random_license_plate()
Generates random license number of the format:
(Does not include alphabets I and O)
[A-Z]{2,2}-[0-9]{2,2}-[A-Z]{2,2}-[0-9]{4,4}
- initialize_vehicles(self)
Initializes vehicles for simulation with random license plate numbers.
- park_all_vehicles(self)
Parks all the vehicles in the parking_lot object.
- remove_parking_for_some_vehicles(self):
  - Generates a random integer less than the total number of vehicles in the list and tries removing that many vehicles from the parking lot.
  - Generates random integer and removes the vehicle present in that index from parking, if they are parked.
- search_vehicle_by_license_plate(self, license_plate)
Searches the vehicle list by license plate and checks their status of parked/ not parked in the parking lot.
"""
class TestParkingLot:
    # Default Constructor
    def __init__(self, distribution_of_vehicles: dict, rows=10, columns=10, height=5):
        print("**" * 30)
        print("Parking Lot Application has Started")
        print("**" * 30 + "\n")
        self.vehicles_count = 0
        self.distribution_of_vehicles = distribution_of_vehicles
        for value in self.distribution_of_vehicles.values():
            self.vehicles_count += value
        print("Total of " + str(self.vehicles_count) + " vehicles are going to be initialized.")
        self.rows = rows
        self.columns = columns
        self.levels = height
        self.vehicle_list = []
        self.initialize_vehicles()
        self.parked = [False for i in range(self.vehicles_count)]
        self.parking_lot = ParkingLot(rows, columns, height)

    @staticmethod
    def generate_random_license_plate():
        # taking template as 2 chars 2 digits 2 chars 4 digits
        string1 = "ABCDEFGHJKLMNPQRSTUVWXYZ"
        string2 = "1234567890"
        len1 = len(string1)
        len2 = len(string2)
        license_number = ""
        for i in range(2):
            index = random.randint(0, len1 - 1)
            license_number += string1[index]
        license_number += "-"
        for i in range(2):
            index = random.randint(0, len2 - 1)
            license_number += string2[index]
        license_number += "-"
        for i in range(2):
            index = random.randint(0, len1 - 1)
            license_number += string1[index]
        license_number += "-"
        for i in range(4):
            index = random.randint(0, len2 - 1)
            license_number += string2[index]
        return license_number

    def initialize_vehicles(self):
        total_initializations = 0
        print("#" * 30)
        for (vehicle_type, count) in self.distribution_of_vehicles.items():
            x = count
            while x != 0:
                vehicle_object = None
                if vehicle_type == VehicleSize.MOTORCYCLE:
                    vehicle_object = Motorcycle(self.generate_random_license_plate())
                elif vehicle_type == VehicleSize.CAR:
                    vehicle_object = Car(self.generate_random_license_plate())
                else:
                    vehicle_object = Bus(self.generate_random_license_plate())
                print("Vehicle " + vehicle_object.vehicle_type.name + " added with license plate " +
                      vehicle_object.license_plate)
                self.vehicle_list.append(vehicle_object)
                total_initializations += 1
                if total_initializations % 5 == 0:
                    print("Initialized " + str(total_initializations) + " vehicles. Sleeping now...")
                    time.sleep(2)
                x -= 1
        random.shuffle(self.vehicle_list)
        print("#" * 30 + "\n")

    def park_all_vehicles(self):
        print("Parking all Vehicles.....")
        for i in range(len(self.vehicle_list)):
            if (i + 1) % 10 == 0:
                print("Taking some rest... Initiated Parking Protocol for "
                      + str(i + 1) + " vehicles till now.")
                time.sleep(2)
            self.parked[i] = self.parking_lot.park_vehicle(self.vehicle_list[i])
        print("." * 30)
        print("Parking Status of All Vehicles")
        print(self.parked)
        print("." * 30 + "\n")

    def remove_parking_for_some_vehicles(self):
        count = random.randint(1, len(self.vehicle_list))
        print(str(count) + " vehicles will be removed from parking from " + str(self.vehicles_count) +
              " vehicles available, if they are parked.")
        i = 0

        while i < count:
            if (i + 1) % 5 == 0:
                print("Taking Rest...")
                time.sleep(2)
            vehicle_index = random.randint(0, len(self.vehicle_list) - 1)
            if self.parked[vehicle_index]:
                self.parking_lot.remove_vehicle(self.vehicle_list[vehicle_index])
                self.parked[vehicle_index] = False
            else:
                print("The Chosen Vehicle " + self.vehicle_list[vehicle_index].vehicle_type.name
                      + " with license plate " +
                      self.vehicle_list[vehicle_index].license_plate + " was not parked initially.\n")
            i += 1

        # self.parking_lot.show_current_status()

    def search_vehicle_by_license_plate(self, license_plate):
        print("*" * 30)
        for vehicle in self.vehicle_list:
            if vehicle.license_plate == license_plate:
                slot_location = vehicle.slot_location
                print(
                    "Vehicle " + vehicle.vehicle_type.name + " with " + vehicle.license_plate +
                    " is occupying the slot " +
                    str(slot_location) +
                    " to " + str((slot_location[0], slot_location[1], slot_location[2] + vehicle.vehicle_type.value)))
        print("*" * 30 + "\n")


In [21]:
# Wrapper function for testing the parking lot system.
def test_parking_lot(vehicle_distribution_list, rows, cols, levels):
    print("The Test Application has begun.")
    f = open("dump.txt" , "w")
    original_stdout = sys.stdout
    sys.stdout = f
    t = TestParkingLot(vehicle_distribution_list, rows, cols, levels)
    t.park_all_vehicles()
    t.remove_parking_for_some_vehicles()
    sys.stdout = original_stdout
    f.close()
    print("The Test Application has ended.")

In [22]:
vehicle_distribution_list = {VehicleSize.MOTORCYCLE: 10, VehicleSize.CAR: 25, VehicleSize.BUS: 15}
rows = 20
cols = 10
levels = 5
test_parking_lot(vehicle_distribution_list, rows, cols, levels)

The Test Application has begun.
The Test Application has ended.
